In [20]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics

import os
import glob

En Colab

In [21]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [22]:
#drive_path = '/content/drive/MyDrive/Colab Notebooks/'
#!ls /content/drive/MyDrive/Colab\ Notebooks/
#folder_path = 'COPERNICUS_GBOV_RM7_20253103525/RM7/'
#file_list = glob.glob(drive_path + folder_path + "*.csv")  # Esto busca todos los archivos que terminen en .csv

En local

In [23]:
# Definimos ruta de los archivos.
datadir_path = os.path.join('..','data')
folder_path = os.path.join(datadir_path, 'COPERNICUS_GBOV_RM7_20253103525/RM7/') # Esto busca todos los archivos que terminen en .csv

In [24]:
file_list = glob.glob(folder_path + "*.csv")  # Esto busca todos los archivos que terminen en .csv

In [25]:
file_list

['..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_001_20220719T190700Z_20220719T190700Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_002_20220720T054300Z_20220720T054300Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_003_20220725T193700Z_20220725T193700Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_004_20220721T053900Z_20220721T053900Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_005_20220722T054400Z_20220722T054400Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_006_20220726T185900Z_20220726T185900Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_007_20220726T190300Z_20220726T190300Z_016_ACR_2.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_BART_BART_010_20220720T190900Z_20220720T190900Z_016_ACR_2.0.csv',
 '..\\da

Primero repasamos el folder y abrimos cada csv para identificar los archivos que cumplen con las columnas requeridas.

In [26]:
# Lista de columnas esperadas
expected_columns = [
    'GBOV_ID', 'Site', 'GROUND_DATA_PI', 'GROUND_DATA_PIs_Email', 'GBOV_Email', 'Network',
    'Elevation', 'IGBP_class', 'Lat_IS', 'Lon_IS', 'TIME_IS', 'Version', 'up_flag', 'down_flag',
    'LAI_Miller_up', 'LAI_Warren_up', 'LAIe_Miller_up', 'LAIe_Warren_up', 'LAI_Miller_down',
    'LAI_Warren_down', 'LAIe_Miller_down', 'LAIe_Warren_down', 'LAI_Miller_up_err', 'LAI_Warren_up_err',
    'LAIe_Miller_up_err', 'LAIe_Warren_up_err', 'clumping_Miller_up', 'clumping_Warren_up',
    'LAI_Miller_down_err', 'LAI_Warren_down_err', 'LAIe_Miller_down_err', 'LAIe_Warren_down_err',
    'clumping_Miller_down', 'clumping_Warren_down', 'clumping_Miller_up_err', 'clumping_Warren_up_err',
    'clumping_Miller_down_err', 'clumping_Warren_down_err'
]

# Lista para guardar los nombres de los archivos que no cumplen
invalid_files = []

# Paso 1: Leer cada archivo CSV y verificar las columnas

for file in file_list:
    df_invidual = pd.read_csv(file, sep=';')

    # Verificar si las columnas son las esperadas
    missing_columns = [col for col in expected_columns if col not in df_invidual.columns]
    extra_columns = [col for col in df_invidual.columns if col not in expected_columns]

    # Si faltan columnas o hay columnas extra, agregamos el nombre del archivo a la lista
    if missing_columns or extra_columns:
        invalid_files.append(file)

# Paso 2: Mostrar los nombres de los archivos que no cumplen
print("Archivos que no cumplen con las columnas esperadas:")
invalid_files


Archivos que no cumplen con las columnas esperadas:


['..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_HARV_HARV_011_20180814T110900Z_20180814T110900Z_021_ACR_1.0.csv',
 '..\\data\\COPERNICUS_GBOV_RM7_20253103525/RM7\\GBOV_RM7_HARV_HARV_024_20180814T203500Z_20180814T203500Z_021_ACR_1.0.csv']

In [27]:
valid_files_list = [archi for archi in file_list if archi not in invalid_files]

Creamos un unico dataframe a partir de todos los csv validos

In [28]:
df_juntos = []
for file in valid_files_list:
    df_invidual = pd.read_csv(file, sep=';')

    # Limpiar los nombres de las columnas
    df_invidual.columns = df_invidual.columns.str.strip()  # Elimina espacios extra en los nombres de las columnas
    if len(df_invidual.columns) != 38:
       print(len(df_invidual.columns))
       print(file)

    df_juntos.append(df_invidual)  # Agregar cada DataFrame a la lista

# Combinar todos los DataFrames en uno solo
df_insitu = pd.concat(df_juntos, axis=0, ignore_index=True)

# Verificar el DataFrame combinado
df_insitu.head()

,GBOV_ID,Site,GROUND_DATA_PI,GROUND_DATA_PIs_Email,GBOV_Email,Network,Elevation,IGBP_class,Lat_IS,Lon_IS,...,LAI_Miller_down_err,LAI_Warren_down_err,LAIe_Miller_down_err,LAIe_Warren_down_err,clumping_Miller_down,clumping_Warren_down,clumping_Miller_up_err,clumping_Warren_up_err,clumping_Miller_down_err,clumping_Warren_down_err
0,GBOV_RM7_958,Bartlett Experimental Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,232,Mixed Forest,44.063901,-71.287308,...,0.038954,0.037536,0.025952,0.027376,0.870976,0.914188,0.020388,0.045145,0.086048,0.119765
1,GBOV_RM7_979,Bartlett Experimental Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,232,Mixed Forest,44.063901,-71.287308,...,0.043254,0.059245,0.03314,0.050986,0.899206,0.927963,0.018524,0.039834,0.069546,0.129738
2,GBOV_RM7_910,Bartlett Experimental Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,232,Mixed Forest,44.063901,-71.287308,...,0.066568,0.067922,0.034103,0.039934,0.848925,0.909968,0.017414,0.039298,0.075073,0.126795
3,GBOV_RM7_934,Bartlett Experimental Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,232,Mixed Forest,44.063901,-71.287308,...,0.017358,0.029335,0.015763,0.026817,0.953405,0.961113,0.019804,0.040532,0.065676,0.131289
4,GBOV_RM7_973,Bartlett Experimental Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,232,Mixed Forest,44.063901,-71.287308,...,0.05235,0.065564,0.041639,0.050919,0.888029,0.906574,0.017908,0.037853,0.075053,0.123742


Hacemos inventario de los sites y coordenadas de los medidas de campo.

In [29]:
df_coord = df_insitu[['Site','Lat_IS', 'Lon_IS']].drop_duplicates()
coordenadas = tuple(zip(df_coord['Site'].values.tolist(), df_coord['Lat_IS'].values.tolist(), df_coord['Lon_IS'].values.tolist()))
coordenadas

(('Bartlett Experimental Forest', 44.063901, -71.287308),
 ('Blandy Experimental Farm', 39.0602607727051, -78.0716400146484),
 ('Lyndon B. Johnson National Grassland', 33.40123, -97.57),
 ('Central Plains Experimental Range', 40.81555, -104.74566),
 ('Dead Lake', 32.54172, -87.80389),
 ('Disney Wilderness Preserve', 28.1250400543213, -81.43625),
 ('Harvard Forest', 42.5377998352051, -72.171501159668),
 ('Jones Ecological Research Center', 31.1948394775391, -84.468777),
 ('Jornada', 32.5907, -106.84261),
 ('Konza Prairie Biological Station', 39.110446, -96.612935),
 ('Lajas Experimental Station', 18.02125, -67.0769),
 ('Lenoir Landing', 31.853861, -88.161181),
 ('Moab', 38.24836, -109.38831),
 ('Niwot Ridge Mountain Research Station', 40.0543, -105.58245),
 ('Onaqui Ault', 40.1775894165039, -112.452438354492),
 ('Oak Ridge', 35.9641189575195, -84.282600402832),
 ('Ordway Swisher Biological Station', 29.6839, -81.9934),
 ('Pu u Maka ala Natural Area Reserve', 19.55309, -155.31731),
 ('Sm

In [30]:
df_insitu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9406 entries, 0 to 9405
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   GBOV_ID                   9406 non-null   object 
 1   Site                      9406 non-null   object 
 2   GROUND_DATA_PI            9176 non-null   object 
 3   GROUND_DATA_PIs_Email     9176 non-null   object 
 4   GBOV_Email                9406 non-null   object 
 5   Network                   9406 non-null   object 
 6   Elevation                 9406 non-null   int64  
 7   IGBP_class                9406 non-null   object 
 8   Lat_IS                    9406 non-null   float64
 9   Lon_IS                    9406 non-null   float64
 10  TIME_IS                   9406 non-null   object 
 11  Version                   9406 non-null   float64
 12  up_flag                   7713 non-null   float64
 13  down_flag                 7713 non-null   float64
 14  LAI_Mill

Grabamos el csv del dataframe insitu crudo

Para Colab

In [31]:
#df_insitu_backup = df_insitu.copy()
#df_insitu.to_csv(drive_path + 'NEON_LAI_USA_2017_2022.csv')

Para local

In [ ]:
#df_insitu.to_csv('../data/NEON_LAI_USA_2017_2024.csv')